In [1]:
setwd("/hpc/hub_oudenaarden/agiladi/data/tuft_release/")

In [2]:
library(devtools)
#source_url("https://raw.githubusercontent.com/aygoldberg/PIC-seq/master/scripts/metacell_functions.r")
source("release_code/metacell_functions.r")
library("Seurat")
library(gtools)
library(metacell)
library(GEOquery)

Loading required package: reshape2
Warning message:
“package ‘reshape2’ was built under R version 3.6.3”Loading required package: plyr
Loading required package: plotrix

Attaching package: ‘metacell’

The following objects are masked _by_ ‘.GlobalEnv’:

    mc_colorize_sup_hierarchy, mcell_mat_rpt_cor_anchors,
    mcell_mc_export_tab

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
  

## Download and extract data generated in this project:

In [4]:
gse_id = "GSE233451"
gse=getGEO(gse_id)
filePaths = getGEOSuppFiles(gse_id)

Found 2 file(s)
GSE233451-GPL18573_series_matrix.txt.gz

── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM7428256 = col_character(),
  GSM7428257 = col_character(),
  GSM7428258 = col_character(),
  GSM7428259 = col_character(),
  GSM7428260 = col_character(),
  GSM7428261 = col_character(),
  GSM8378150 = col_character(),
  GSM8378151 = col_character()
)
File stored at: 
/scratch/23222958/RtmpodXHAK/GPL18573.soft
GSE233451-GPL24676_series_matrix.txt.gz

── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM7428262 = col_character()
)
File stored at: 
/scratch/23222958/RtmpodXHAK/GPL24676.soft


In [5]:
raw_path = grep("RAW", rownames(filePaths), v=T)
umidir = "release_umi.tab/"
dir.create(umidir)
untar(raw_path, exdir = umidir)

In [6]:
setwd(umidir)
#system("gzip -d *.gz")
system("ls -1 | awk -F'_' '{print $0,$2}' | xargs -n 2 mv")
setwd("..")

In [7]:
metadata_fn = grep("metadata", rownames(filePaths), v=T)
cell_metadata = read.delim(metadata_fn, stringsAsFactors = F, row.names = 1)
head(cell_metadata)

,amp_batch_id,cell_barcode,umicount,mt_count,coord,source,type,gating,treatment,age,clover
HUB.JO.s037_AAACAGGC,HUB.JO.s037,AAACAGGC,7296,1280,H18,Ileum,organoid,AVIL-Clover+,no_cyto,adult,AVIL
HUB.JO.s037_AAAGCGGA,HUB.JO.s037,AAAGCGGA,3461,626,P12,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL
HUB.JO.s037_AAAGGCTG,HUB.JO.s037,AAAGGCTG,4062,1019,O10,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL
HUB.JO.s037_AACACGCA,HUB.JO.s037,AACACGCA,8352,1471,L20,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL
HUB.JO.s037_AACATGGG,HUB.JO.s037,AACATGGG,4986,692,P10,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL
HUB.JO.s037_AACCCAAC,HUB.JO.s037,AACCCAAC,8453,1449,O15,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL


### Create a unified UMI matrix from all SORT-seq data:

In [8]:
umi_files = list.files(umidir, pattern = "*.txt")
data_list = lapply(paste0(umidir, "/", umi_files), read.delim, stringsAsFactor=F, row.names=1)

In [9]:
names(data_list) = vecsplit(umi_files, "\\.", 1)

In [10]:
genes = names(table(unlist(lapply(data_list, rownames))))

In [11]:
cells = unlist(sapply(data_list, colnames))

In [12]:
data = matrix(0, nrow = length(genes), ncol = length(cells), dimnames = list(genes, cells))
for (samp in names(data_list)) {
    X = data_list[[samp]]
    shared = intersect(genes, rownames(X))
    message(samp, ": ", length(shared), " genes and ", ncol(X), " cells")    
    #data[ shared, paste0(samp, "_", colnames(X))] = as.matrix(X[shared,])
    data[ shared, colnames(X)] = as.matrix(X[shared,])
}

HUB-JO-s037: 25267 genes and 384 cells
HUB-JO-s042: 25502 genes and 384 cells
HUB-JO-s043: 24643 genes and 380 cells
HUB-JO-s054: 26898 genes and 383 cells
HUB-JO-s057: 25126 genes and 384 cells
HUB-JO-s058: 22283 genes and 384 cells
HUB-LH-s002: 25742 genes and 384 cells
HUB-LH-s003: 24955 genes and 384 cells


In [6]:
dir.create("saved_work/")
scdb_init("saved_work/", force_reinit=T)
id = "tuft"
id_all = "tuft_all"

Warning message in dir.create("saved_work/"):
“'saved_work' already exists”initializing scdb to saved_work/


In [14]:
scdb_add_mat(id_all, tgScMat(mat=data, cell_metadata=cell_metadata[ colnames(data),]))

In [16]:
bad_genes = unique(c(grep("ERCC|^MT-", rownames(data), v=T)))

In [17]:
# Import the metacell structure of tuft-enriched organoids
import_metacell_structure(id = "ileum_organoids", 
                        folder = "release_import/figure2_cs2/", 
                        all_id = "tuft_all", 
                        bad_genes = bad_genes, 
                        url = NULL)

add batch counts
compute footprints
compute absolute ps
compute coverage ps
sim graph is missing 0 nodes, out of 953


## Generate a filtered UMI table of the 10x data (organoids seeded from tuft cells)

In [19]:
data = Seurat::Read10X(data.dir = umidir)

10X data contains more than one type and is being returned as a list containing matrices of each type.


In [20]:
cellplex = as.matrix(data[[2]])

### Filter based on the number of Cellplex UMI per cell

In [22]:
first = apply(cellplex, 2, max)
second = apply(cellplex, 2, function(x) sort(x,T)[2])

In [23]:
y = log2(first + 1)
z = log2((first + 1) / (second + 1))

good_cells = names(which(y > 6 & z > 3)) # More than 2^6 UMI per cells and more than 2^3-fold difference between top and second cell-plex tag

In [24]:
data = as.matrix(data[[1]][,good_cells])

In [25]:
tag = rownames(cellplex)[ max.col(t(cellplex))]
names(tag) = colnames(cellplex)

In [27]:
cellplex_md = read.delim("release_import/figure3_10x/tuft_10x_metadata.txt", stringsAsFactors = F, row.names = 1)
cellplex_md

,seeder_cell,medium
CMO309,non-tuft,expansion
CMO310,tuft,diff
CMO311,tuft,diff+IL4/13
CMO312,tuft,expansion
CMO301,non-tuft,diff
CMO302,non-tuft,diff+IL4/13


In [28]:
cell_metadata = cbind(tag = tag[ good_cells], cellplex_md[ tag[ good_cells],], umicount = colSums(data[,good_cells]))
rownames(cell_metadata) = good_cells
head(cell_metadata)

,tag,seeder_cell,medium,umicount
AAACCCAAGCACGGAT,CMO312,tuft,expansion,9961
AAACCCAAGGATATAC,CMO302,non-tuft,diff+IL4/13,5519
AAACCCAAGTCTACCA,CMO301,non-tuft,diff,8189
AAACCCAAGTTGGAAT,CMO311,tuft,diff+IL4/13,2714
AAACCCACACTCGATA,CMO311,tuft,diff+IL4/13,6549
AAACCCACATGGCTAT,CMO312,tuft,expansion,8552


In [29]:
colnames(data) = paste0(colnames(data), "-1")
rownames(cell_metadata) = paste0(rownames(cell_metadata), "-1")

In [7]:
id = "10x"
id_all = "10x_all"

In [31]:
scdb_add_mat(id_all, tgScMat(mat=data[, rownames(cell_metadata)], cell_metadata=cell_metadata))

In [4]:
bad_genes = unique(c(grep("^MT-|^ERCC|^IG[HLK]", rownames(data), v=T)))

In [13]:
# Import the metacell structure of tuft-seeded organoids
import_metacell_structure(id = id, 
                        folder = "release_import/figure3_10x/", 
                        all_id = id_all, 
                        bad_genes = bad_genes, 
                        url = NULL)

add batch counts
compute footprints
compute absolute ps
compute coverage ps


## Download and organize published data

In [36]:
# Elmentaite's data

setwd("release_umi.tab/")
system("wget https://cellgeni.cog.sanger.ac.uk/gutcellatlas/Full_obj_raw_counts_nosoupx_v2.h5ad")
setwd("..")

In [ ]:
library(SeuratDisk)

Convert("release_umi.tab/Full_obj_raw_counts_nosoupx_v2.h5ad", dest="h5seurat", overwrite=T)
obj = LoadH5Seurat("release_umi.tab/Full_obj_raw_counts_nosoupx_v2.h5seurat")
save(obj, file = "release_umi.tab/Full_obj_raw_counts_nosoupx_v2.Rda")


In [3]:
load("release_umi.tab/Full_obj_raw_counts_nosoupx_v2.Rda")

In [4]:
X = obj@assays$RNA
metadata = obj@meta.data

In [6]:
good_cells = rownames(metadata)[metadata$Diagnosis == "Healthy adult" & metadata$Region == "SmallInt" & metadata$category == "Epithelial"]

In [8]:
id = "Elmentaite"
id_all = "Elmentaite_all"

scdb_init("saved_work/", force_reinit=T)
scdb_add_mat(id_all, tgScMat(mat=X[, good_cells], cell_metadata=metadata[ good_cells,]))

initializing scdb to saved_work/


In [9]:
bad_genes = unique(c(grep("^MT-|^ERCC|^IG[HLK]", rownames(X), v=T)))


In [10]:
import_metacell_structure(id = id, 
                        folder = "release_import/figure1_Elmentaite/", 
                        all_id = id_all, 
                        bad_genes = bad_genes, 
                        url = NULL)

add batch counts
compute footprints
compute absolute ps
compute coverage ps
sim graph is missing 0 nodes, out of 15184


In [8]:
# Hickey's data
links = read.delim("release_import/figure1_Hickey/links.txt", stringsAsFactors = F, h=F, skip = 1)
data_files = grep("\\.gz", links[,"V2"])
links$folder = "./"
links[ data_files, "folder"] = vecsplit(links[ data_files, "V2"], "_", 1)
links$file = links$V2
links[ data_files, "file"] = vecsplit(links[ data_files, "V2"], "_", 2)

In [21]:
curdir = getwd()
umidir = paste0(curdir, "/release_umi.tab/figure1_Hickey")
umidir
dir.create(umidir)

[1] "/hpc/hub_oudenaarden/agiladi/data/tuft_release/release_umi.tab/figure1_Hickey"

Warning message in dir.create(umidir):
“'/hpc/hub_oudenaarden/agiladi/data/tuft_release/release_umi.tab/figure1_Hickey' already exists”

In [ ]:
## This block can download all the data from Hickey et al. into specified directories (one for each dataset).
## Notice this doesn't always work in R. If not, you can use the download_wrapper.sh in shell instead.


setwd(umidir)

p = function(i, links) {
    message(links[i, "V2"])
    dir.create(links[i, "folder"], showWarnings = F)
    cmd1 = paste0(curdir, "/release_code/download_wrapper.sh ", links[i, "V3"], " ", links[i, "V1"], " ", links[i, "folder"], "/", links[i, "file"])
    #message(cmd1)
    system(cmd1)
    Sys.sleep(1)
}

invisible(sapply(seq_len(nrow(links)), p, links))

setwd(curdir)

In [22]:
setwd(curdir)
umidir = "release_umi.tab/figure1_Hickey/"
files = list.files(umidir)


In [24]:
metadata_fn = grep("Epithelial", files, v=T)
metadata_list = lapply(paste0(umidir, "/", metadata_fn), read.delim, stringsAsFactors = F, row.names = 1)
head(metadata_list[[1]])

,UMAP_1,UMAP_2,CellType
B001-A-301_AAACCCACAGCTATTG-1,2.292168,-3.326214,TA1
B001-A-301_AAACCCAGTGTTCCAA-1,4.247404,6.580899,Enterocytes
B001-A-301_AAACGAAGTGGTATGG-1,4.615074,5.245791,Immature Enterocytes
B001-A-301_AAACGAATCCAATGCA-1,4.477911,-0.670288,TA1
B001-A-301_AAACGAATCGTGGCGT-1,0.353033,-3.104455,TA1
B001-A-301_AAACGCTAGACGATAT-1,5.073505,6.059951,Enterocytes


In [25]:
l = sapply(metadata_list, nrow)

In [26]:
metadata = do.call(rbind, metadata_list)

In [27]:
metadata$tissue = rep(vecsplit(metadata_fn, "_", 1), l)
metadata$folder = vecsplit(rownames(metadata), "_", 1)
head(metadata)

,UMAP_1,UMAP_2,CellType,tissue,folder
B001-A-301_AAACCCACAGCTATTG-1,2.292168,-3.326214,TA1,Colon,B001-A-301
B001-A-301_AAACCCAGTGTTCCAA-1,4.247404,6.580899,Enterocytes,Colon,B001-A-301
B001-A-301_AAACGAAGTGGTATGG-1,4.615074,5.245791,Immature Enterocytes,Colon,B001-A-301
B001-A-301_AAACGAATCCAATGCA-1,4.477911,-0.670288,TA1,Colon,B001-A-301
B001-A-301_AAACGAATCGTGGCGT-1,0.353033,-3.104455,TA1,Colon,B001-A-301
B001-A-301_AAACGCTAGACGATAT-1,5.073505,6.059951,Enterocytes,Colon,B001-A-301


In [26]:
tenxruns = names(table(metadata$folder))

In [28]:
tenxfiles = t(sapply(tenxruns, function(x) setdiff(list.files(paste0(umidir, "/", x)), "")))

In [30]:
tuft_cells = rownames(metadata)[ metadata$CellType == "Tuft" & metadata$tissue == "Jejunum"]

In [31]:
rel_folders = names(table(metadata[ tuft_cells, "folder"]))

In [ ]:
data_list = lapply(rel_folders, function(f) {
    data = Seurat::Read10X(data.dir = paste0(umidir, "/", f))
    if (length(data) == 2) {data = data[[1]]}
    colnames(data) = paste0(f, "_", colnames(data))
    X = as.matrix(data[, intersect( tuft_cells, colnames(data))])
    message(f, ": ", nrow(X), " genes over ", ncol(X), " cells")
    X
})

In [ ]:
genes = names(table(unlist(lapply(data_list, rownames))))

In [ ]:
cells = unlist(sapply(data_list, colnames))

In [ ]:
names(data_list) = rel_folders

In [ ]:
data = matrix(0, nrow = length(genes), ncol = length(cells), dimnames = list(genes, cells))

In [ ]:
for (samp in names(data_list)) {
    X = data_list[[samp]]
    message(samp, ": ", nrow(X), " genes and ", ncol(X), " cells")    
    data[ rownames(X), colnames(X)] = as.matrix(X)
}

In [31]:
data = as.matrix(data[, intersect(rownames(metadata), colnames(data))])

In [33]:
dir.create("saved_work/")
scdb_init("saved_work/", force_reinit=T)
id = "hickey_tuft"
id_all = "hickey_tuft"

Warning message in dir.create("saved_work/"):
“'saved_work' already exists”initializing scdb to saved_work/


In [34]:
scdb_add_mat(id_all, tgScMat(mat=data, cell_metadata=metadata[ colnames(data),]))